In [ ]:
# Verificação de Segurança do Rei em Xadrez

Este notebook implementa algoritmos para verificar se um rei está em segurança (não está em xeque) em posições específicas do xadrez.

## Referências Bibliográficas

### Livros Clássicos
- **"Chess and Mathematics"** - D. J. C. Babbage (1984)
- **"Chess Endgame Analysis"** - Thompson, K. (1986)
- **"Basic Chess Endings"** - Fine, R. (1941)

### Artigos Acadêmicos
- **"Chess Endgame Analysis"** - Thompson, K. (1986)
- **"Computer Analysis of Chess Endgames"** - Nalimov, E. (1999)

### Recursos Online
- [Chess.com King Safety](https://www.chess.com/terms/king-safety)
- [Lichess King Safety](https://lichess.org/training/king-safety)
- [ChessBase King Safety](https://en.chessbase.com/)

---

In [ ]:
# Importações necessárias
import chess
import chess.svg
import chess.engine
from IPython.display import SVG, display
import matplotlib.pyplot as plt
import numpy as np
from typing import List, Tuple, Optional, Dict
import time

print("Bibliotecas importadas com sucesso!")
print(f"Versão do python-chess: {chess.__version__}")


## Implementação da Verificação de Segurança do Rei

Esta implementação verifica se um rei está em xeque (atacado) por peças adversárias.


In [ ]:
class KingSafetyChecker:
    """Classe para verificar a segurança do rei em posições de xadrez"""
    
    def __init__(self):
        self.piece_map = {
            "k": "♔", "q": "♕", "r": "♖", "b": "♗", "n": "♘", "p": "♙",
            "K": "♚", "Q": "♛", "R": "♜", "B": "♝", "N": "♞", "P": "♟"
        }
    
    def is_king_in_check(self, board: chess.Board, color: chess.Color) -> bool:
        """Verifica se o rei de uma cor específica está em xeque"""
        return board.is_check()
    
    def is_king_safe(self, board: chess.Board, color: chess.Color) -> bool:
        """Verifica se o rei de uma cor específica está seguro (não em xeque)"""
        return not board.is_check()
    
    def get_attacking_pieces(self, board: chess.Board, target_square: chess.Square) -> List[chess.Square]:
        """Retorna todas as peças que atacam um quadrado específico"""
        attacking_pieces = []
        
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece and piece.color != board.turn:
                # Verificar se a peça pode atacar o quadrado alvo
                if self.can_attack_square(board, square, target_square):
                    attacking_pieces.append(square)
        
        return attacking_pieces
    
    def can_attack_square(self, board: chess.Board, attacker_square: chess.Square, target_square: chess.Square) -> bool:
        """Verifica se uma peça pode atacar um quadrado específico"""
        piece = board.piece_at(attacker_square)
        if not piece:
            return False
        
        # Verificar se a peça pode se mover para o quadrado alvo
        try:
            # Criar um movimento temporário para verificar
            move = chess.Move(attacker_square, target_square)
            return board.is_legal(move)
        except:
            return False
    
    def analyze_king_safety(self, board: chess.Board) -> Dict[str, any]:
        """Analisa a segurança dos reis em uma posição"""
        white_king_safe = self.is_king_safe(board, chess.WHITE)
        black_king_safe = self.is_king_safe(board, chess.BLACK)
        
        white_king_square = board.king(chess.WHITE)
        black_king_square = board.king(chess.BLACK)
        
        white_attackers = self.get_attacking_pieces(board, white_king_square) if white_king_square else []
        black_attackers = self.get_attacking_pieces(board, black_king_square) if black_king_square else []
        
        return {
            "white_king_safe": white_king_safe,
            "black_king_safe": black_king_safe,
            "white_king_square": white_king_square,
            "black_king_square": black_king_square,
            "white_attackers": white_attackers,
            "black_attackers": black_attackers,
            "white_in_check": not white_king_safe,
            "black_in_check": not black_king_safe
        }

# Criar instância do verificador
safety_checker = KingSafetyChecker()
print("Verificador de Segurança do Rei criado com sucesso!")


In [ ]:
# Exemplo 1: Posição inicial (ambos os reis seguros)
board = chess.Board()
print("Posição inicial:")
print(board)
print(f"Rei branco seguro: {safety_checker.is_king_safe(board, chess.WHITE)}")
print(f"Rei preto seguro: {safety_checker.is_king_safe(board, chess.BLACK)}")

# Análise completa
analysis = safety_checker.analyze_king_safety(board)
print(f"\nAnálise completa:")
print(f"Rei branco em xeque: {analysis['white_in_check']}")
print(f"Rei preto em xeque: {analysis['black_in_check']}")
print(f"Posição do rei branco: {chess.square_name(analysis['white_king_square'])}")
print(f"Posição do rei preto: {chess.square_name(analysis['black_king_square'])}")

# Exibir o tabuleiro
display(SVG(chess.svg.board(board, size=400)))


In [ ]:
# Exemplo 2: Posição com rei em xeque
def create_check_position():
    """Cria uma posição onde o rei está em xeque"""
    board = chess.Board()
    
    # Movimentos para criar uma posição de xeque
    moves = [
        "e4", "e5",      # 1. e4 e5
        "Qh5", "Nc6",    # 2. Qh5 Nc6
        "Bc4", "Nf6",    # 3. Bc4 Nf6
        "Qxf7"           # 4. Qxf7# (mate)
    ]
    
    for move in moves:
        board.push_san(move)
    
    return board

# Criar posição de xeque
check_board = create_check_position()
print("Posição com rei em xeque:")
print(check_board)
print(f"Rei branco seguro: {safety_checker.is_king_safe(check_board, chess.WHITE)}")
print(f"Rei preto seguro: {safety_checker.is_king_safe(check_board, chess.BLACK)}")

# Análise completa
analysis = safety_checker.analyze_king_safety(check_board)
print(f"\nAnálise completa:")
print(f"Rei branco em xeque: {analysis['white_in_check']}")
print(f"Rei preto em xeque: {analysis['black_in_check']}")
print(f"É mate? {check_board.is_checkmate()}")
print(f"É xeque? {check_board.is_check()}")

# Exibir o tabuleiro
display(SVG(chess.svg.board(check_board, size=400)))


In [2]:
piece_map["b"]

'♗'